In [1]:
import random
import numpy as np
import torch
import binvox_rw
import config as cfg
import grakel
import inspect
import copy
from deap import base
from deap import creator
from deap import tools
from generate import SuperGenerator
from utils import generate_z
from evolution import mutate_mask,cluster_graph
G=SuperGenerator()
pop_size=100

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("n_iter", random.choice, (4,5,6))
toolbox.register("num_clusters", random.choice, (10,20,100,200,250,300))
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.n_iter,toolbox.num_clusters),n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
def generate_voxels():
    voxels=[]
    for i in range(10):        
        voxels.append(G.generate(generate_z(), cfg.model)) 
    return voxels

def evalOneMax(individual,G,voxels):
    print(individual)
    if individual[0]==0 or individual[1]==0 :
        return 0,
    graphs=cluster_graph(voxels,num_clusters=individual[1])

    G_grakel=grakel.graph_from_networkx(graphs, node_labels_tag='label')
    gk=grakel.WeisfeilerLehman(n_iter=individual[0],normalize=True)
    K,base_kernel=gk.fit_transform(G_grakel)
    var=np.var(K)
    print(var)
    return var,

toolbox.register("evaluate", evalOneMax,G=G)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


def main():
    random.seed(64)
    
    pop = toolbox.population(n=pop_size)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40
    
    print("Start of evolution")
    
    voxels=generate_voxels()
    fitnesses = []
    for i in range(len(pop)):
            fitnesses.append(toolbox.evaluate(pop[i],voxels=voxels))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))
    
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        try:
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            voxels=generate_voxels()
            fitnesses=[]
            for i in range(len(invalid_ind)):
                fitnesses.append(toolbox.evaluate(invalid_ind[i],voxels=voxels))
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
        except AssertionError:
            pass
        
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        pop[:] = offspring
        
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

if __name__ == "__main__":
    main()

Start of evolution
[5, 10]
num_clusters: 17
num_clusters: 21
num_clusters: 17
num_clusters: 19
num_clusters: 20
num_clusters: 33
num_clusters: 10
num_clusters: 14
num_clusters: 25
num_clusters: 17
0.04010650116032407
[6, 250]
num_clusters: 433
num_clusters: 546
num_clusters: 445
num_clusters: 485
num_clusters: 509
num_clusters: 845
num_clusters: 261
num_clusters: 374
num_clusters: 628
num_clusters: 444
0.0038403026892236667
[5, 250]
num_clusters: 433
num_clusters: 546
num_clusters: 445
num_clusters: 485
num_clusters: 509
num_clusters: 845
num_clusters: 261
num_clusters: 374
num_clusters: 628
num_clusters: 444
0.0036144641591538762
[6, 10]
num_clusters: 17
num_clusters: 21
num_clusters: 17
num_clusters: 19
num_clusters: 20
num_clusters: 33
num_clusters: 10
num_clusters: 14
num_clusters: 25
num_clusters: 17
0.044461771711233854
[4, 100]
num_clusters: 173
num_clusters: 218
num_clusters: 178
num_clusters: 194
num_clusters: 203
num_clusters: 338
num_clusters: 104
num_clusters: 149
num_clust

num_clusters: 533
0.003092427580636295
[4, 20]
num_clusters: 34
num_clusters: 43
num_clusters: 35
num_clusters: 38
num_clusters: 40
num_clusters: 67
num_clusters: 20
num_clusters: 29
num_clusters: 50
num_clusters: 35
0.021132080093084295
[6, 100]
num_clusters: 173
num_clusters: 218
num_clusters: 178
num_clusters: 194
num_clusters: 203
num_clusters: 338
num_clusters: 104
num_clusters: 149
num_clusters: 251
num_clusters: 177
0.006574670667664781
[5, 200]
num_clusters: 347
num_clusters: 437
num_clusters: 356
num_clusters: 388
num_clusters: 407
num_clusters: 676
num_clusters: 209
num_clusters: 299
num_clusters: 502
num_clusters: 355
0.004061172812501111
[5, 300]
num_clusters: 520
num_clusters: 655
num_clusters: 534
num_clusters: 582
num_clusters: 611
num_clusters: 1014
num_clusters: 313
num_clusters: 449
num_clusters: 753
num_clusters: 533
0.0032765739229351477
[5, 20]
num_clusters: 34
num_clusters: 43
num_clusters: 35
num_clusters: 38
num_clusters: 40
num_clusters: 67
num_clusters: 20
num

num_clusters: 177
0.005716949024213322
[6, 300]
num_clusters: 520
num_clusters: 655
num_clusters: 534
num_clusters: 582
num_clusters: 611
num_clusters: 1014
num_clusters: 313
num_clusters: 449
num_clusters: 753
num_clusters: 533
0.0034743350602610278
[4, 10]
num_clusters: 17
num_clusters: 21
num_clusters: 17
num_clusters: 19
num_clusters: 20
num_clusters: 33
num_clusters: 10
num_clusters: 14
num_clusters: 25
num_clusters: 17
0.034891340058221025
[5, 300]
num_clusters: 520
num_clusters: 655
num_clusters: 534
num_clusters: 582
num_clusters: 611
num_clusters: 1014
num_clusters: 313
num_clusters: 449
num_clusters: 753
num_clusters: 533
0.0032765739229351477
[5, 300]
num_clusters: 520
num_clusters: 655
num_clusters: 534
num_clusters: 582
num_clusters: 611
num_clusters: 1014
num_clusters: 313
num_clusters: 449
num_clusters: 753
num_clusters: 533
0.0032765739229351477
[5, 10]
num_clusters: 17
num_clusters: 21
num_clusters: 17
num_clusters: 19
num_clusters: 20
num_clusters: 33
num_clusters: 10

num_clusters: 52
num_clusters: 46
num_clusters: 38
num_clusters: 28
num_clusters: 39
num_clusters: 33
num_clusters: 46
num_clusters: 40
0.023529863131334913
[6, 200]
num_clusters: 330
num_clusters: 831
num_clusters: 524
num_clusters: 466
num_clusters: 386
num_clusters: 284
num_clusters: 393
num_clusters: 335
num_clusters: 465
num_clusters: 402
0.006143495588169826
[5, 10]
num_clusters: 16
num_clusters: 41
num_clusters: 26
num_clusters: 23
num_clusters: 19
num_clusters: 14
num_clusters: 19
num_clusters: 16
num_clusters: 23
num_clusters: 20
0.04061878467276876
[4, 10]
num_clusters: 16
num_clusters: 41
num_clusters: 26
num_clusters: 23
num_clusters: 19
num_clusters: 14
num_clusters: 19
num_clusters: 16
num_clusters: 23
num_clusters: 20
0.036092998319364564
[6, 20]
num_clusters: 33
num_clusters: 83
num_clusters: 52
num_clusters: 46
num_clusters: 38
num_clusters: 28
num_clusters: 39
num_clusters: 33
num_clusters: 46
num_clusters: 40
0.027005225302557688
[6, 10]
num_clusters: 16
num_clusters

num_clusters: 83
num_clusters: 52
num_clusters: 46
num_clusters: 38
num_clusters: 28
num_clusters: 39
num_clusters: 33
num_clusters: 46
num_clusters: 40
0.023529863131334913
[5, 10]
num_clusters: 16
num_clusters: 41
num_clusters: 26
num_clusters: 23
num_clusters: 19
num_clusters: 14
num_clusters: 19
num_clusters: 16
num_clusters: 23
num_clusters: 20
0.04061878467276876
[5, 10]
num_clusters: 16
num_clusters: 41
num_clusters: 26
num_clusters: 23
num_clusters: 19
num_clusters: 14
num_clusters: 19
num_clusters: 16
num_clusters: 23
num_clusters: 20
0.04061878467276876
  Evaluated 64 individuals
  Min 0.0038403026892236667
  Max 0.04450746361276437
  Avg 0.03016646017949026
  Std 0.013889715579821027
-- Generation 1 --
[6, 10]
num_clusters: 19
num_clusters: 20
num_clusters: 25
num_clusters: 16
num_clusters: 18
num_clusters: 22
num_clusters: 22
num_clusters: 15
num_clusters: 18
num_clusters: 28
0.04495198097607874
[6, 10]
num_clusters: 19
num_clusters: 20
num_clusters: 25
num_clusters: 16
num

...

0.043777715447496465
[6, 10]
num_clusters: 15
num_clusters: 26
num_clusters: 18
num_clusters: 25
num_clusters: 21
num_clusters: 27
num_clusters: 16
num_clusters: 16
num_clusters: 23
num_clusters: 23
0.043777715447496465
[6, 10]
num_clusters: 15
num_clusters: 26
num_clusters: 18
num_clusters: 25
num_clusters: 21
num_clusters: 27
num_clusters: 16
num_clusters: 16
num_clusters: 23
num_clusters: 23
0.043777715447496465
[6, 10]
num_clusters: 15
num_clusters: 26
num_clusters: 18
num_clusters: 25
num_clusters: 21
num_clusters: 27
num_clusters: 16
num_clusters: 16
num_clusters: 23
num_clusters: 23
0.043777715447496465
[6, 10]
num_clusters: 15
num_clusters: 26
num_clusters: 18
num_clusters: 25
num_clusters: 21
num_clusters: 27
num_clusters: 16
num_clusters: 16
num_clusters: 23
num_clusters: 23
0.043777715447496465
[6, 10]
num_clusters: 15
num_clusters: 26
num_clusters: 18
num_clusters: 25
num_clusters: 21
num_clusters: 27
num_clusters: 16
num_clusters: 16
num_clusters: 23
num_clusters: 23
0.043

num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 2

num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 36
num_clusters: 25
num_clusters: 14
num_clusters: 24
num_clusters: 19
num_clusters: 17
num_clusters: 13
0.0446410685781412
[6, 10]
num_clusters: 24
num_clusters: 21
num_clusters: 14
num_clusters: 3

num_clusters: 21
num_clusters: 17
num_clusters: 14
num_clusters: 22
num_clusters: 15
num_clusters: 18
num_clusters: 15
0.04489709782712725
[6, 10]
num_clusters: 22
num_clusters: 11
num_clusters: 19
num_clusters: 21
num_clusters: 17
num_clusters: 14
num_clusters: 22
num_clusters: 15
num_clusters: 18
num_clusters: 15
0.04489709782712725
[6, 10]
num_clusters: 22
num_clusters: 11
num_clusters: 19
num_clusters: 21
num_clusters: 17
num_clusters: 14
num_clusters: 22
num_clusters: 15
num_clusters: 18
num_clusters: 15
0.04489709782712725
[6, 10]
num_clusters: 22
num_clusters: 11
num_clusters: 19
num_clusters: 21
num_clusters: 17
num_clusters: 14
num_clusters: 22
num_clusters: 15
num_clusters: 18
num_clusters: 15
0.04489709782712725
[6, 10]
num_clusters: 22
num_clusters: 11
num_clusters: 19
num_clusters: 21
num_clusters: 17
num_clusters: 14
num_clusters: 22
num_clusters: 15
num_clusters: 18
num_clusters: 15
0.04489709782712725
[6, 10]
num_clusters: 22
num_clusters: 11
num_clusters: 19
num_cluste

KeyboardInterrupt: 

In [ ]:
individual=[4, 2, 100, 0.7, 5000, [32, 0, 32]]
print(type(individual[5])(not individual[5]))